In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import warnings

warnings.filterwarnings("ignore")

# ✅ Step 1: Load Excel File and NormalizedData sheet
file_path = "/kaggle/input/crime-data-district-wise/Crime_Data_District_Wise.xlsx"
sheet_name = "NormalizedData"

df = pd.read_excel(file_path, sheet_name=sheet_name)

# ✅ Step 2: Clean Data
df = df[['DISTRICT', 'CrimeType', 'YEAR', 'Count']]
df = df.dropna()
df = df[df['YEAR'] <= 2012]  # Use only historical data for forecasting

# ✅ Step 3: Forecast Setup
forecast_years = list(range(2013, 2021))  # Forecasting for 2013–2020
forecast_data = []

# ✅ Step 4: Forecasting loop per (District, CrimeType)
for (district, crime), group in df.groupby(['DISTRICT', 'CrimeType']):
    group = group.sort_values(by='YEAR')
    
    if group['Count'].sum() == 0 or len(group) < 3:
        continue  # Skip if there's too little data to learn from

    try:
        model = ExponentialSmoothing(group['Count'], trend='add', seasonal=None)
        fitted_model = model.fit()
        forecast = fitted_model.forecast(len(forecast_years))
        
        for year, predicted_count in zip(forecast_years, forecast):
            forecast_data.append({
                "DISTRICT": district,
                "CrimeType": crime,
                "YEAR": year,
                "Count": round(predicted_count)
            })
    
    except Exception as e:
        print(f"Skipped {district} - {crime} due to error: {e}")

# ✅ Step 5: Convert to DataFrame
forecast_df = pd.DataFrame(forecast_data)

# ✅ Step 6: Save forecast to CSV
output_path = "/kaggle/working/crime_forecast_2013_2020.csv"
forecast_df.to_csv(output_path, index=False)

# ✅ Preview results
forecast_df.head(10)



,DISTRICT,CrimeType,YEAR,Count
0,24 PARGANAS NORTH,ARSON,2013,32
1,24 PARGANAS NORTH,ARSON,2014,35
2,24 PARGANAS NORTH,ARSON,2015,38
3,24 PARGANAS NORTH,ARSON,2016,42
4,24 PARGANAS NORTH,ARSON,2017,45
5,24 PARGANAS NORTH,ARSON,2018,48
6,24 PARGANAS NORTH,ARSON,2019,51
7,24 PARGANAS NORTH,ARSON,2020,54
8,24 PARGANAS NORTH,ASSAULT ON WOMEN WITH INTENT TO OUTRAGE HER MO...,2013,217
9,24 PARGANAS NORTH,ASSAULT ON WOMEN WITH INTENT TO OUTRAGE HER MO...,2014,230
